In [10]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.io as pio

# Descargar los datos históricos del S&P 500 desde Yahoo Finance
sp500 = yf.download('^GSPC', start='1900-01-01')

# Calcular los rendimientos diarios
sp500['Daily Return'] = sp500['Adj Close'].pct_change()

# Separar los datos por años
sp500['Year'] = sp500.index.year
annual_data = {year: data for year, data in sp500.groupby('Year')}

# Calcular la evolución de una inversión X a principios de cada año
def investment_evolution(year_data, initial_investment=1000):
    return initial_investment * (1 + year_data['Daily Return']).cumprod()

investment_evolutions = {year: investment_evolution(data) for year, data in annual_data.items() if not data.empty}

# Hacer un plot interactivo de todos los años con el mismo punto de inicio
fig = go.Figure()
for year, evolution in investment_evolutions.items():
    evolution_normalized = (evolution / evolution.iloc[0]) * 100  # Normalizar para que todos los años empiecen en el mismo punto (100%)
    if year == 2024:
        fig.add_trace(go.Scatter(x=evolution_normalized.index.day_of_year, y=evolution_normalized.values,
                                 mode='lines', name=str(year), line=dict(width=2, color='blue')))
    else:
        fig.add_trace(go.Scatter(x=evolution_normalized.index.day_of_year, y=evolution_normalized.values,
                                 mode='lines', name=str(year), line=dict(color='gray', width=1), opacity=0.5))

fig.update_layout(title='Rendimiento acumulado del S&P 500 por año (normalizado desde 100%)',
                  xaxis_title='Día del año',
                  yaxis_title='Rendimiento acumulado (%)',
                  legend_title='Año',
                  template='plotly_white')

pio.show(fig)

# Calcular el rendimiento anualizado de cada año
def annualized_return(year_data):
    total_return = (1 + year_data['Daily Return']).prod() - 1
    num_days = len(year_data)
    return (1 + total_return) ** (252 / num_days) - 1

annual_returns = {year: annualized_return(data) for year, data in annual_data.items() if not data.empty}

# Crear un DataFrame con los rendimientos anualizados
df_annual_returns = pd.DataFrame(list(annual_returns.items()), columns=['Year', 'Annualized Return'])

# Ordenar los años por rendimiento anualizado y asignar posiciones en el ranking
df_annual_returns.sort_values(by='Annualized Return', ascending=False, inplace=True)
df_annual_returns['Rank'] = range(1, len(df_annual_returns) + 1)

# Calcular el porcentaje en el que nos encontramos en 2024
current_rank = df_annual_returns[df_annual_returns['Year'] == 2024]['Rank'].values[0]
total_years = len(df_annual_returns)
percentage_position = current_rank / total_years

# Mostrar la tabla de rendimientos anualizados y ranking
print(df_annual_returns)
print(f"\nEn el año 2024 nos encontramos en el {percentage_position:.2%} del ranking de rendimientos historicos.")

[*********************100%***********************]  1 of 1 completed


    Year  Annualized Return  Rank
6   1933           0.462656     1
27  1954           0.450222     2
8   1935           0.417605     3
1   1928           0.382370     4
31  1958           0.380595     5
..   ...                ...   ...
3   1930          -0.285803    94
47  1974          -0.296201    95
81  2008          -0.383675    96
10  1937          -0.388305    97
4   1931          -0.470665    98

[98 rows x 3 columns]

En el año 2024 nos encontramos en el 12.24% del ranking de rendimientos historicos.
